In [73]:
%matplotlib inline

from music21 import pitch
import sqlite3
import sqlalchemy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pomegranate import *


In [4]:
name = 'wjazzd.db'
jazzdb = sqlite3.connect(name)
cursor = jazzdb.cursor()
cursor.execute("SELECT name FROM sqlite_master;")
tables = cursor.fetchall()
tables

[(u'melody',),
 (u'sqlite_sequence',),
 (u'beats',),
 (u'sections',),
 (u'melody_type',),
 (u'transcription_info',),
 (u'solo_info',),
 (u'record_info',),
 (u'track_info',),
 (u'composition_info',),
 (u'esac_info',),
 (u'popsong_info',),
 (u'db_info',),
 (u'melind',),
 (u'esacind',),
 (u'soloind',),
 (u'transind',),
 (u'trackind',),
 (u'compind',),
 (u'sectind',),
 (u'typeind',)]

In [195]:
engine = sqlalchemy.create_engine("sqlite:///%s" % name, execution_options={"sqlite_raw_colnames": True})
melody = pd.read_sql_table('melody', engine)
beats = pd.read_sql_table('beats', engine)
solo_info = pd.read_sql_table('solo_info', engine)
record_info = pd.read_sql_table('record_info', engine)
composition_info = pd.read_sql_table('composition_info', engine)

In [197]:
melody.loc[melody['melid'] == 5]
print 'melody columns'
print melody.columns.values

melody_f = melody.filter(['eventid', 'melid', 'pitch'])
#solo_info['key'].value_counts().plot(kind='bar')

melody columns
['eventid' 'melid' 'onset' 'pitch' 'duration' 'period' 'division' 'bar'
 'beat' 'tatum' 'subtatum' 'num' 'denom' 'beatprops' 'beatdur'
 'tatumprops' 'f0_mod' 'loud_max' 'loud_med' 'loud_sd' 'loud_relpos'
 'loud_cent' 'loud_s2b' 'f0_range' 'f0_freq_hz' 'f0_med_dev']


In [200]:
transpose_dest_midi = pitch.Pitch('B-').midi

# find the distance to destination key
# returns (dist, minor/maj) for usable songs or None for unusable songs
def get_key_transpose(s):
    try:
        n = s.split('-')[0]
        m = s.split('-')[1]
        if m not in ['maj', 'min']:
            return None
        if n[-1] == 'b':
            n = n[:-1] + '-'
        note_midi = pitch.Pitch(n).midi

        # move to relative major of minor key
        if m == 'min':
            mode = 'minor'
            note_midi += 3
        else:
            mode = 'major'
            
        # find smallest distance to the destination pitch
        dist = transpose_dest_midi - note_midi
        if dist > 6:
            dist -= 12
        elif dist < -6:
            dist += 12
        return dist, mode
    
    except:
        if s == '':
            return None
        elif s == 'Ab':
            return 2, 'major'
        else:
            return None

In [201]:
transposition_dists = {}
for i, r in solo_info.iterrows():
    transposition_dists[r['melid']] = get_key_transpose(r['key'])


In [167]:
# melody.apply(lambda x: transposition_dists.get(x['melid']) is not None)
# m2 = melody[melody.apply(lambda x: transposition_dists[x['melid']] is not None, axis=1)]
# melody.apply(lambda x: x['pitch'] += transposition_dists[x['melid']][0], axis=1)

SyntaxError: invalid syntax (<ipython-input-167-043151ad33a5>, line 3)

In [217]:
melody_filtered = melody.copy()
notes_by_melody = {}
for k, v in transposition_dists.iteritems():
    if v is None:
        melody_filtered = melody_filtered.loc[melody_filtered['melid'] != k]
    else:
        melody_filtered.loc[melody_filtered['melid'] == k, 'pitch'] += v[0]
        notes_by_melody[k] = melody_filtered.loc[melody_filtered['melid'] == k]
print 'filtered melody df, from %s to %s' % (melody.shape[0], melody_filtered.shape[0])

filtered melody df, from 200809 to 180597


In [220]:
for k, v in notes_by_melody.iteritems():
    notes_by_melody[k] = notes_by_melody[k].copy().filter(['eventid', 'pitch'])

In [224]:
notes_by_melody[1]

,eventid,pitch
0,1,65.0
1,2,63.0
2,3,58.0
3,4,61.0
4,5,63.0
5,6,58.0
6,7,58.0
7,8,50.0
8,9,57.0
9,10,60.0
